In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a60b9a8a831-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0a60b9a8a831-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[6]: True

In [0]:
# list the topics stored on the mounted S3 bucket
display(dbutils.fs.ls("/mnt/user-0a60b9a8a831-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-0a60b9a8a831-bucket/topics/0a60b9a8a831.geo/,0a60b9a8a831.geo/,0,1702836448283
dbfs:/mnt/user-0a60b9a8a831-bucket/topics/0a60b9a8a831.pin/,0a60b9a8a831.pin/,0,1702836448283
dbfs:/mnt/user-0a60b9a8a831-bucket/topics/0a60b9a8a831.user/,0a60b9a8a831.user/,0,1702836448283


In [0]:
topics = dbutils.fs.ls("/mnt/user-0a60b9a8a831-bucket/topics")

# creates dataframes based on topic names
for item in topics:
  topic = (item.name)
  df_name = item.name.split(".", 1)[1].split("/", 1)[0]

  # Asterisk(*) indicates reading all the content of the specified file that have .json extension
  file_location = f"/mnt/user-0a60b9a8a831-bucket/topics/{topic}partition=0/*.json"
  file_type = "json"
  # Ask Spark to infer the schema
  infer_schema = "true"
  # Read in JSONs from mounted S3 bucket and dynamically generated dataframe name
  exec(f"df_{df_name} = spark.read.format(file_type).option('inferSchema',infer_schema).load(file_location)")

  # Display Spark dataframe to check its content
  display(eval(f"df_{df_name}"))

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03 05:41:59
Dominican Republic,9979,14.9967,-120.682,2018-07-18 19:01:46
Cote d'Ivoire,2923,-84.6302,-164.507,2019-09-08 22:53:09
French Guiana,8304,-28.8852,-164.87,2019-09-13 04:50:29
French Guiana,8304,-28.8852,-164.87,2019-09-13 04:50:29
Cote d'Ivoire,4315,-45.8508,66.1003,2019-12-15 03:51:28
Azerbaijan,5069,-63.0063,-157.474,2021-03-20 09:32:44
Mozambique,6145,-65.9079,-143.845,2019-12-05 02:09:44


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
travel,"This Costa Rica itinerary is the ultimate guide to spending two weeks in Costa Rica. Find out about visiting La Fortuna, Arenal, Monteverde, Naranjo, Corcovado National Park, Or…",1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
christmas,Here are the best DIY Christmas Centerpieces ideas perfect for your Christmas & holiday season home decor. From Christmas Vignettes to Table Centerpieces.,1,500k,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,2418,image,HikenDip,Local save in /data/christmas,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,da8745a6-5160-46c4-877d-181d50a729fd
travel,"Are you traveling to Paris during the summer? Find out what to do in Paris, France during the summer. Fun summertime activities in Paris. Enjoy the incredible outdoors when trav…",1,3k,https://i.pinimg.com/originals/6c/4c/90/6c4c90bba27ebf8c8bfe4c1acfb9f07a.jpg,9979,image,Petite in Paris,Local save in /data/travel,"Torre Eiffel Paris,Tour Eiffel,Picnic In Paris,Hello France,Voyage Europe,Destination Voyage,Beautiful Places To Travel,Travel Aesthetic,Paris Travel",Paris in the Summer. 10 fun things to do in Paris in the Summertime • Petite in Paris,2b2abc85-fc51-481f-8ae6-17681993da28
home-decor,"Holiday mantle decor, Christmas decor, metallic mercury glass style Christmas trees, eucalyptus vine, evergreen pine branches, white neutral holiday decor, cozy mantle for the h…",1,83k,https://i.pinimg.com/originals/9d/82/1a/9d821a80acd8f90c16454e978bd9b115.jpg,6145,image,Stylin by Aylin,Local save in /data/home-decor,"Winter Home Decor,Christmas Living Room Decor,Living Room Decor Cozy,Christmas Decor,Cozy Fireplace,Rustic Fireplace Decor,Fireplace Decorations,Rustic Room,House Decorations",HOLIDAY MANTLE DECOR - @AMAZON & @TARGET FINDS,82e13a07-db99-43a3-b1c0-89a4b75821da
education,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel jo

age,date_joined,first_name,ind,last_name
21,2015-11-10 09:27:42,Andrea,8731,Alexander
36,2015-12-20 16:38:13,Michelle,4315,Prince
32,2017-10-10 20:09:33,Christian,10625,Lang
32,2016-04-02 03:51:23,Brittany,1313,Jones
34,2016-12-22 00:02:02,Thomas,10794,Turner
20,2016-01-07 19:49:22,David,2959,Griffith
26,2015-12-20 10:28:00,Brendan,9875,Joseph
21,2016-01-03 15:42:12,Annette,2074,Forbes
25,2015-12-28 04:21:39,Charles,8304,Berry
23,2015-11-25 13:36:22,Corey,6063,Andrews


In [0]:
# Unmount the S3 bucket
dbutils.fs.unmount("/mnt/user-0a60b9a8a831-bucket")